# Muestreo representantivo de los datos

### Determinar cantidad de bloques (recorrido de una línea) por trip de cada unidad

In [1]:
from pathlib import Path
import pandas as pd, numpy as np, gc

In [2]:
# Obtener lista de archivos con características listas

FEATS_DIR = Path("D:/2025/UVG/Tesis/repos/backend/features_ready_without_idle_rows")
files = sorted(FEATS_DIR.glob("*_features.parquet"))
assert files, "No hay archivos en features_ready_without_idle_rows/*.parquet"

In [ ]:
from pathlib import Path
import pandas as pd, numpy as np, gc

# 1) Carga columnas mínimas necesarias (incluye LINEA y Fecha)
cols_min = ["Placa", "trip_id", "block_id", "LINEA", "Fecha"]
df = pd.concat(
    [pd.read_parquet(f, columns=cols_min) for f in files],
    ignore_index=True
)

# 2) Limpieza y tipos
df = df.dropna(subset=["Placa", "trip_id", "block_id"])
df["Fecha"] = pd.to_datetime(df["Fecha"], errors="coerce")
df = df.dropna(subset=["Fecha"])  # si vas a calcular t0/t1
df["LINEA"] = df["LINEA"].astype("string").fillna("<NA>")

# 3) Identificador único de bloque (global)
df["unique_block_id"] = (
    df["Placa"].astype("string") + "#" +
    df["trip_id"].astype("string") + "#" +
    df["block_id"].astype("string")
)

# (Opcional pero MUY recomendable) Verificar que LINEA sea constante dentro del bloque
consistency = (
    df.groupby("unique_block_id")["LINEA"]
      .nunique(dropna=False)
      .rename("lineas_en_block")
)
inconsistent_blocks = consistency[consistency > 1].index
if len(inconsistent_blocks):
    print(f"ADVERTENCIA: {len(inconsistent_blocks)} blocks tienen >1 LINEA. "
          "Revisar generación de block_id.")

# 4) Metadata por bloque
block_meta = (
    df.sort_values("Fecha")
      .groupby("unique_block_id")
      .agg(
          Placa=("Placa","first"),
          trip_id=("trip_id","first"),
          block_id=("block_id","first"),
          LINEA=("LINEA","first"),   # debería ser constante en el bloque
          n=("unique_block_id","size"),
          t0=("Fecha","min"),
          t1=("Fecha","max")
      )
      .reset_index()
)

print(f"Blocks totales: {len(block_meta):,} | filas totales: {int(block_meta['n'].sum()):,}")

# Sanity check: #blocks por trip (por Placa+trip_id)
trip_counts = (
    df.groupby(["Placa","trip_id"])["block_id"]
      .nunique()
)
print("Promedio de blocks por trip:", float(trip_counts.mean()),
      "| # trips con >1 block:", int((trip_counts > 1).sum()))


Blocks totales: 58,174 | filas totales: 43,030,926
Promedio de blocks por trip: 1.0702800161901609 | # trips con >1 block: 2526


In [4]:
block_meta.head()

,unique_block_id,Placa,trip_id,block_id,LINEA,n,t0,t1
0,100#1#1,100,1,1,Linea_12,816,2024-01-01 06:11:07,2024-01-01 19:01:49
1,100#10#1,100,10,1,Linea_12,910,2024-01-11 05:24:46,2024-01-11 20:20:45
2,100#100#1,100,100,1,Linea_12,565,2024-04-19 12:25:41,2024-04-19 18:59:20
3,100#101#1,100,101,1,Linea_12,421,2024-04-20 15:12:56,2024-04-20 21:16:34
4,100#102#1,100,102,1,Linea_12,908,2024-04-22 06:46:22,2024-04-23 04:03:41


In [5]:
# Filas por línea
line_row_counts = (
    df.groupby("LINEA")["Placa"]
      .count()
      .rename("n_filas")
      .reset_index()
)

print('Filas por línea:')
print(line_row_counts)

Filas por línea:
        LINEA   n_filas
0     Linea_1   2138693
1    Linea_12  18864271
2  Linea_13-A   9778243
3  Linea_13-B     80323
4  Linea_18-A   4818382
5  Linea_18-B    834939
6     Linea_2    549587
7     Linea_6   5460214
8     Linea_7    506274


In [6]:
# Bloques por línea
line_block_counts = (
    df.groupby("LINEA")["unique_block_id"]
      .nunique()
      .rename("n_blocks")
      .reset_index()
)
print('Blocks por línea:')
print(line_block_counts)

Blocks por línea:
        LINEA  n_blocks
0     Linea_1      3307
1    Linea_12     26118
2  Linea_13-A     11398
3  Linea_13-B       175
4  Linea_18-A      6477
5  Linea_18-B      2018
6     Linea_2      1150
7     Linea_6      6830
8     Linea_7       701


### Definición de cuotas por línea

In [7]:
import numpy as np
import pandas as pd

def compute_line_quotas(block_meta: pd.DataFrame,
                        sample_rows: int,
                        min_per_line: int = 10_000,
                        smooth: str = "none",      # "none" o "sqrt"
                        allow_overshoot: bool = True  # no recortar si se pasa
                       ) -> pd.Series:
    # Capacidad real por línea (máximo seleccionable)
    line_capacity = block_meta.groupby("LINEA")["n"].sum().astype(int)

    # 1) Base para proporción
    base = np.sqrt(line_capacity) if smooth == "sqrt" else line_capacity

    # 2) Cuota proporcional preliminar
    prop = (sample_rows * (base / base.sum())).round().astype(int)

    # 3) Mínimo garantizado
    min_floor = pd.Series(min_per_line, index=line_capacity.index, dtype="int64")
    quota = pd.Series(np.maximum(prop, min_floor), index=line_capacity.index, dtype="int64")

    # 4) Respetar capacidad
    quota = quota.clip(upper=line_capacity)

    # 5) Ajuste para alcanzar EXACTAMENTE sample_rows (opcional).
    #    Si allow_overshoot=True, solo rellenar cuando falta; si sobra, dejar así.
    def adjust_to_target(quota: pd.Series, capacity: pd.Series, target: int, floor: pd.Series) -> pd.Series:
        quota = quota.astype(float)
        current = int(quota.sum())

        if current == target:
            return quota.round().astype(int)

        if current > target:
            if allow_overshoot:
                # No recortamos: devolvemos tal cual (redondeado)
                return quota.round().astype(int)
            else:
                # (opcional) recorte proporcional por encima del floor
                reducible = (quota - floor).clip(lower=0.0)
                excess = current - target
                if reducible.sum() > 0:
                    factor = max(0.0, 1.0 - excess / reducible.sum())
                    quota = floor + reducible * factor
                quota = quota.round()
                diff = int(target - quota.sum())
                if diff < 0:
                    order = (quota - floor).sort_values(ascending=False).index
                    for ln in order:
                        if diff == 0: break
                        if quota[ln] > floor[ln]:
                            quota[ln] -= 1
                            diff += 1
                return quota.clip(lower=floor, upper=capacity).round().astype(int)

        # current < target:
        
        # repartir faltante según capacidad remanente
        """ leftover = target - current
        rem_cap = (capacity - quota).clip(lower=0.0)
        if rem_cap.sum() > 0:
            add = np.floor(leftover * (rem_cap / rem_cap.sum()))
            quota = quota + add
            quota = quota.round()
            diff = int(target - quota.sum())
            if diff > 0:
                order = (capacity - quota).sort_values(ascending=False).index
                for ln in order:
                    if diff == 0: break
                    if quota[ln] < capacity[ln]:
                        quota[ln] += 1
                        diff -= 1 """

        # retorno definitivo
        return quota.clip(lower=floor, upper=capacity).round().astype(int)

    # Caso extremo: suma de mínimos > target
    sum_min = int(min_floor.sum())
    if sum_min > sample_rows:
        nL = len(min_floor)
        base = sample_rows // nL
        quota = pd.Series(base, index=min_floor.index, dtype="int64")
        remainder = sample_rows - base * nL
        for ln in min_floor.index[:remainder]:
            quota[ln] += 1
        return quota.clip(upper=line_capacity).astype(int)

    # Ajuste normal (respetando allow_overshoot)
    quota = adjust_to_target(quota, line_capacity, sample_rows, min_floor)
    return quota


In [8]:
SAMPLE_ROWS = 5_000_000
MIN_PER_LINE = 10_000       # el mínimo garantizado por línea
SMOOTH = "sqrt"            # "none" para proporcional puro; "sqrt" para suavizar
ALLOW_OVERSHOOT = True  # no recortar si se pasa

line_quota = compute_line_quotas(block_meta, SAMPLE_ROWS, MIN_PER_LINE, smooth=SMOOTH, allow_overshoot=ALLOW_OVERSHOOT)

print("Cuotas por línea:\n", line_quota.sort_values(ascending=False))
print("Suma final:", int(line_quota.sum()))

Cuotas por línea:
 LINEA
Linea_12      1347632
Linea_13-A     970245
Linea_6        725030
Linea_18-A     681086
Linea_1        453759
Linea_18-B     283517
Linea_2        230022
Linea_7        220772
Linea_13-B      80323
dtype: int32
Suma final: 4992386


### Selección de bloques según cuotas

In [9]:
import random
import pandas as pd
from collections import defaultdict

In [10]:
def sample_blocks_by_line(block_meta: pd.DataFrame,
                          line_quota: pd.Series,
                          seed: int = 25,
                          max_rel_overshoot_per_line: float = 0.10  # tolera +10% por línea
                         ):
    """
    Devuelve:
      chosen_blocks: Index de unique_block_id seleccionados
      per_line_stats: DataFrame con cuota, seleccionado y over/under por línea
    """
    rng = random.Random(seed)

    # Asegura tipos y columnas
    assert {"unique_block_id","LINEA","n"}.issubset(block_meta.columns), "Faltan columnas en block_meta"
    # Subset solo líneas con cuota > 0
    target_lines = line_quota[line_quota > 0].index
    bm = block_meta[block_meta["LINEA"].isin(target_lines)].copy()

    # Barajar orden de bloques por línea
    idxs_by_line = {}
    for ln, sub in bm.groupby("LINEA"):
        idxs = list(sub.index)
        rng.shuffle(idxs)
        idxs_by_line[ln] = idxs

    chosen = []
    taken_per_line = defaultdict(int)

    # Greedy: llenar por línea
    for ln in target_lines:
        quota = int(line_quota[ln])
        if quota <= 0: 
            continue
        acc = 0
        for i in idxs_by_line[ln]:
            n_i = int(bm.at[i, "n"])
            bid = bm.at[i, "unique_block_id"]
            chosen.append(bid)
            acc += n_i
            if acc >= quota:
                break
        taken_per_line[ln] = acc

    chosen = pd.Index(chosen).unique()
    chosen_meta = bm[bm["unique_block_id"].isin(chosen)].copy()
    keys_df = chosen_meta[["Placa","trip_id","block_id"]].drop_duplicates().reset_index(drop=True)

    # Stats finales
    chosen_meta = bm[bm["unique_block_id"].isin(chosen)].copy()
    
    selected_by_line = chosen_meta.groupby("LINEA")["n"].sum()
    per_line = pd.DataFrame({
        "quota": line_quota.astype(int),
        "selected": selected_by_line.astype(int)
    }).fillna(0).astype(int)
    per_line["diff"] = per_line["selected"] - per_line["quota"]

    return chosen, per_line, keys_df

In [11]:
chosen_blocks, per_line_stats, keys_df = sample_blocks_by_line(block_meta, line_quota, seed=25)
print(per_line_stats.sort_values("selected", ascending=False).head(10))
print("Filas totales seleccionadas:", int(per_line_stats["selected"].sum()))

              quota  selected  diff
LINEA                              
Linea_12    1347632   1347834   202
Linea_13-A   970245    970476   231
Linea_6      725030    725071    41
Linea_18-A   681086    682399  1313
Linea_1      453759    453762     3
Linea_18-B   283517    283820   303
Linea_2      230022    230817   795
Linea_7      220772    220959   187
Linea_13-B    80323     80323     0
Filas totales seleccionadas: 4995461


### Materializar la muestra

In [12]:
NEEDED_COLS = [
    "Placa","trip_id","block_id",
    "LINEA","DIR","proxima_est_teorica","Fecha",
    "dist_a_prox_m","dist_estacion_m","vel_mps","Altitud (m)","s_m","dist_m",
    "time_diff","dwell_same_xy_s","is_no_progress","progress_event","hour","dow",
    "is_weekend","is_peak","ETA_proxima_est_s"
]

parts = []
for f in files:
    # lee solo columnas disponibles de este parquet (intersección)
    with_cols = [c for c in NEEDED_COLS if c in pd.read_parquet(f, columns=[c for c in NEEDED_COLS]).columns]
    df = pd.read_parquet(f, columns=with_cols)

    # join por llaves → filtra solo filas de bloques elegidos
    df = df.merge(keys_df, on=["Placa","trip_id","block_id"], how="inner")
    if not df.empty:
        parts.append(df)

sample_df = pd.concat(parts, ignore_index=True) if parts else pd.DataFrame(columns=NEEDED_COLS)
print("Muestra final:", len(sample_df), "filas")

Muestra final: 4995461 filas


In [13]:
# Verificar distribución por línea en la muestra final
final_line_counts = (
    sample_df.groupby("LINEA")["Placa"]
      .count()
      .rename("n_filas")
      .reset_index()
)
print("Distribución final por línea en la muestra:")
print(final_line_counts.sort_values("n_filas", ascending=False))

Distribución final por línea en la muestra:
        LINEA  n_filas
1    Linea_12  1347834
2  Linea_13-A   970476
7     Linea_6   725071
4  Linea_18-A   682399
0     Linea_1   453762
5  Linea_18-B   283820
6     Linea_2   230817
8     Linea_7   220959
3  Linea_13-B    80323


In [14]:
# Guardar muestra final
OUTPUT_PATH = Path("D:/2025/UVG/Tesis/repos/backend/features_sampled_without_idle_rows/sample_features.parquet")

# 1) Definir columnas por tipo
str_cols = ["Placa","trip_id","block_id","LINEA","DIR","proxima_est_teorica"]
dt_cols  = ["Fecha"]
float_cols = [
    "dist_a_prox_m","dist_estacion_m","vel_mps","Altitud (m)","s_m","dist_m",
    "time_diff","dwell_same_xy_s","ETA_proxima_est_s"
]
int_cols = ["hour","dow"]
boolish_cols = ["is_no_progress","progress_event","is_weekend","is_peak"]

# 2) Casts seguros
for c in str_cols:
    if c in sample_df:
        sample_df[c] = sample_df[c].astype("string")

for c in dt_cols:
    if c in sample_df:
        sample_df[c] = pd.to_datetime(sample_df[c], errors="coerce")

for c in float_cols:
    if c in sample_df:
        sample_df[c] = pd.to_numeric(sample_df[c], errors="coerce").astype("float32")

for c in int_cols:
    if c in sample_df:
        sample_df[c] = pd.to_numeric(sample_df[c], errors="coerce").astype("Int16")  # nullable int

for c in boolish_cols:
    if c in sample_df:
        sample_df[c] = pd.to_numeric(sample_df[c], errors="coerce").astype("Int8")   # 0/1 con NA soportado

# 3) (Opcional) verifica tipos
print(sample_df.dtypes)

# 4) Guardar
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)
sample_df.to_parquet(OUTPUT_PATH, index=False, engine="pyarrow")

Placa                  string[python]
trip_id                string[python]
block_id               string[python]
LINEA                  string[python]
DIR                    string[python]
proxima_est_teorica    string[python]
Fecha                  datetime64[ns]
dist_a_prox_m                 float32
dist_estacion_m               float32
vel_mps                       float32
Altitud (m)                   float32
s_m                           float32
dist_m                        float32
time_diff                     float32
dwell_same_xy_s               float32
is_no_progress                   Int8
progress_event                   Int8
hour                            Int16
dow                             Int16
is_weekend                       Int8
is_peak                          Int8
ETA_proxima_est_s             float32
dtype: object


In [15]:
# Mostrar el DataFrame final
print(sample_df.head())

  Placa trip_id block_id     LINEA  DIR proxima_est_teorica  \
0    49       1        1  Linea_12  IDA         MONTE MARÍA   
1    49       1        1  Linea_12  IDA         MONTE MARÍA   
2    49       1        1  Linea_12  IDA         MONTE MARÍA   
3    49       1        1  Linea_12  IDA         MONTE MARÍA   
4    49       1        1  Linea_12  IDA         MONTE MARÍA   

                Fecha  dist_a_prox_m  dist_estacion_m  vel_mps  ...  \
0 2024-01-12 09:38:23    2507.478516       361.775177      0.0  ...   
1 2024-01-12 09:39:23    2507.478516       361.775177      0.0  ...   
2 2024-01-12 09:40:23    2507.478516       361.775177      0.0  ...   
3 2024-01-12 09:41:23    2507.478516       361.775177      0.0  ...   
4 2024-01-12 09:42:23    2507.478516       361.775177      0.0  ...   

      dist_m  time_diff  dwell_same_xy_s  is_no_progress  progress_event  \
0  320.33194       60.0              0.0               0               0   
1  320.33194       60.0             60.0  